In [1]:
import tensorflow as tf
import time

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784;

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k * max_pool2_k)) * (input_height//(max_pool1_k * max_pool2_k)) * n_conv2

In [4]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k,  n_conv1, n_conv2])),
    "whl" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo" :  tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bhl" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out]))
}

In [5]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])
    

In [6]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape = [-1, input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['whl']), biases['bhl'])
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    output =  tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output    

In [7]:
x = tf.placeholder(tf.float32, [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
pred = cnn(x, weights, biases)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
batch_size = 100
n_iter= 25
for i in range(n_iter):
    st = time.time()
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimize], feed_dict={x:batch_x , y:batch_y})
        total_cost += c
        
    et = time.time()
    print(i, " : " ,total_cost)   
    print("time taken = ", et-st)
    

0  :  1049459.5101902485
time taken =  1215.1936571598053
1  :  24969.416761532426
time taken =  1210.4437413215637
2  :  16640.180826028503
time taken =  1207.0629963874817
3  :  13367.15996559143
time taken =  1211.226830482483
4  :  13086.194018294402
time taken =  1190.7286517620087
5  :  9705.222378583232
time taken =  1123.3372340202332
6  :  9506.323432174015
time taken =  1123.1924891471863
7  :  9718.012200150471
time taken =  1112.6439638137817
8  :  8032.293064624369
time taken =  1146.25466132164
9  :  8106.544316835708
time taken =  1210.0052506923676
10  :  6786.651739187532
time taken =  1219.008462190628
11  :  5053.228711103957
time taken =  1220.30086684227
12  :  4177.147550846359
time taken =  1214.1342322826385
13  :  4260.666970903039
time taken =  1221.5585889816284
14  :  4410.452014725201
time taken =  1223.5518703460693
15  :  3744.152033457411
time taken =  1224.8778808116913
16  :  3742.7307025676782
time taken =  1224.4708139896393
17  :  3067.661882526737


In [12]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions, correct_preds = sess.run([predictions, correct_predictions], feed_dict = {x:mnist.test.images, y:mnist.test.labels})
correct_preds.sum()

9864